In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

!pip install keras_wrn
import keras_wrn

import pandas as pd
import numpy as np

from sklearn import metrics

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Construct a tf.data.Dataset
train_images, train_info = tfds.load(
    'cifar100', 
    split='train[:80%]', 
    shuffle_files=True, 
    with_info=True, 
    as_supervised=True)

dev_images, dev_info = tfds.load(
    'cifar100', 
    split='train[-20%:]', 
    shuffle_files=True, 
    with_info=True, 
    as_supervised=True)

test_images, test_info = tfds.load(
    'cifar100', 
    split='test', 
    shuffle_files=True, 
    with_info=True, 
    as_supervised=True)


In [ ]:

fig = tfds.show_examples(dev_images, dev_info)


In [ ]:
input_shape = train_info.features["image"].shape
num_classes = train_info.features["label"].num_classes 
print("Image shape: ", input_shape)
print("Labels:\n", train_info.features["label"].names)
print("Number of Labels: ", num_classes)

In [ ]:
def normalize_image(image, label):

  norm_image = tf.cast(image, tf.float32) / 255.0

  return norm_image, label

In [ ]:

batch_size = 100

train_images = train_images.map(normalize_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_images = train_images.cache()
train_images = train_images.shuffle(train_info.splits['train'].num_examples).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

dev_images = dev_images.map(normalize_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch_size)
dev_images = dev_images.cache()
dev_images = dev_images.prefetch(tf.data.experimental.AUTOTUNE)

test_images = test_images.map(normalize_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch_size)
test_images = test_images.cache()
test_images = test_images.prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:

model = keras_wrn.build_model(
    input_dims=input_shape, 
    output_dim=num_classes,
    n=40,
    k=2
    )

"""
model = tf.keras.Sequential([
                             
      tf.keras.layers.Conv2D(32, 5, strides=1, padding='same', activation=tf.nn.leaky_relu, input_shape=input_shape),
      tf.keras.layers.Conv2D(32, 5, strides=2, padding='same', activation=tf.nn.leaky_relu),
      tf.keras.layers.Conv2D(64, 5, strides=1, padding='same', activation=tf.nn.leaky_relu),
      tf.keras.layers.Conv2D(64, 5, strides=2, padding='same', activation=tf.nn.leaky_relu),
      tf.keras.layers.Conv2D(128, 7, strides=1, padding='valid', activation=tf.nn.leaky_relu),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dropout(0.4),
      tf.keras.layers.Dense(100, activation="softmax"),                   
])
"""

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

print(model.summary())


In [ ]:
model.fit(train_images, validation_data=dev_images, epochs=250)

In [ ]:

test_loss, test_accuracy = model.evaluate(test_images)

print(f'Test Loss: {test_loss}; Test Accuracy: {test_accuracy}')


In [ ]:
pred_labels = model.predict(test_images)
pred_labels = np.argmax(pred_labels, axis=1)
print(pred_labels)

In [ ]:
test_labels = np.array([labels.numpy() for images, labels in test_images])
test_labels = np.concatenate(test_labels, axis=None)

print(test_labels)


In [ ]:

PROJ_DIR = '/content/drive/My Drive/data/ml'
out_file = f'{PROJ_DIR}/cnn_sub10.csv'

report = metrics.classification_report(test_labels, pred_labels, output_dict=True)

df = pd.DataFrame(report).transpose()

df.to_csv(out_file, index=False)
